In [24]:
# mine the links for all the match for specific team
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
def mine_match_url(matches_link):
    # get url
    url = matches_link
    match = re.match(r'^(https?://[^/]+)', url)
    if match:
        home_url = match.group(1)
        print(home_url)
    else:
        print("URL not valid")
        return []
    
    try:
        # Ignore SSL certificate errors and set timeout
        match_url_page = requests.get(matches_link, verify=False, timeout=10)
        match_url_page.raise_for_status()  # Raises HTTPError for bad responses
        match_url_page_soup = BeautifulSoup(match_url_page.content, "html.parser")
    except requests.RequestException as e:
        print(f"Failed to fetch match URLs from {matches_link}. Error: {e}")
        return []
    
    # Locate the match section and extract links
    match_sections = match_url_page_soup.find_all("div", class_="d-block d-md-flex")
    scorecard_links = []
    for section in match_sections:
        scorecard_buttons = section.find_all("a", class_="link-scorecard d-none d-md-inline-block rounded-circle")
        for button in scorecard_buttons:
            link = button.get('href')
            if link:
                full_link = home_url + link + "/print"
                scorecard_links.append(full_link)
    
    return scorecard_links

In [25]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def mine_match_data(scorecard_link):
    
    # Add retry mechanism for the request
    max_retries = 3
    for attempt in range(max_retries):
        try:
            scorecard_page = requests.get(scorecard_link, timeout=10)
            scorecard_page.raise_for_status()
            break  # Exit the retry loop if the request is successful
        except requests.RequestException as e:
            print(f"Error fetching data from {scorecard_link}: {e}")
            if attempt < max_retries - 1:
                print(f"Retrying... ({attempt + 1}/{max_retries})")
                time.sleep(5)  # Wait 5 seconds before retrying
            else:
                print("Max retries exceeded. Skipping this match.")
                return None

    scorecard_page_soup = BeautifulSoup(scorecard_page.content, "html.parser")
    
    # Retrieve match information
    match_info_tag = scorecard_page_soup.find('h3', class_='result_main_heading')
    if match_info_tag is None:
        print(f"Error: Could not find match info in {scorecard_link}")
        return None
    
    match_info = match_info_tag.get_text(strip=True)
    try:
        home_team, away_team = match_info.split('Vs')
        home_team = home_team.strip()
        away_team = away_team.strip()
    except ValueError:
        print(f"Error: Unexpected format in match info: {match_info} at {scorecard_link}")
        return None

    # Retrieve score information
    points_details = scorecard_page_soup.find('table', class_='points_details')
    if points_details is None:
        print(f"Error: Could not find points details in {scorecard_link}")
        return None

    try:
        home_score = points_details.find_all('td')[2].get_text(strip=True)
        away_score = points_details.find_all('td')[3].get_text(strip=True)
    except IndexError:
        print(f"Error: Unable to extract scores from points details in {scorecard_link}")
        return None

    # Skip matches with an invalid score of -0 and print the link
    if home_score == '-0' or away_score == '-0':
        print(f"Skipping match with invalid score -0: {scorecard_link}")
        return None

    # Retrieve ground information
    ground_info = None
    fixture_details = scorecard_page_soup.find('table', class_='fixture_details')
    if fixture_details:
        for row in fixture_details.find_all('tr'):
            cells = row.find_all('td')
            if len(cells) > 1 and cells[0].get_text(strip=True).startswith('Ground'):
                ground_info = cells[1].get_text(strip=True)
                break

    if not ground_info:
        print(f"Error: Could not find ground info in {scorecard_link}")
        return None

    def clean_text(text):
        return text.replace('\xa0', ' ').strip()

    # Retrieve batting data
    def get_batting_data(innings_id):
        batting_data = []
        innings_div = scorecard_page_soup.find('div', id=innings_id)
        if innings_div:
            batting_table = innings_div.find('table', class_='batting')
            if batting_table:
                for row in batting_table.find_all('tr')[1:]:  # Skip header row
                    players = row.find_all('td')
                    if len(players) >= 7:
                        player_data = {
                            'Name': clean_text(players[0].get_text(strip=True)),
                            'How Out': clean_text(players[1].get_text(strip=True)),
                            'Bowler': clean_text(players[2].get_text(strip=True)),
                            'Runs': clean_text(players[3].get_text(strip=True)),
                            '4s': clean_text(players[4].get_text(strip=True)),
                            '6s': clean_text(players[5].get_text(strip=True)),
                            'Balls': clean_text(players[6].get_text(strip=True)),
                        }
                        batting_data.append(player_data)
        return batting_data if batting_data else None

    # Retrieve bowling data
    def get_bowling_data(innings_id):
        bowling_data = []
        innings_div = scorecard_page_soup.find('div', id=innings_id)
        if innings_div:
            bowling_table = innings_div.find('table', class_='bowling')
            if bowling_table:
                for row in bowling_table.find_all('tr')[1:]:  # Skip header row
                    players = row.find_all('td')
                    if len(players) >= 7:
                        bowler_data = {
                            'Bowler': clean_text(players[0].get_text(strip=True)),
                            'Overs': clean_text(players[1].get_text(strip=True)),
                            'Maidens': clean_text(players[2].get_text(strip=True)),
                            'Runs': clean_text(players[3].get_text(strip=True)),
                            'Wickets': clean_text(players[4].get_text(strip=True)),
                            'Wides': clean_text(players[5].get_text(strip=True)),
                            'No Balls': clean_text(players[6].get_text(strip=True)),
                        }
                        bowling_data.append(bowler_data)
        return bowling_data if bowling_data else None

    # Retrieve batting and bowling data for both teams
    home_batting_data = get_batting_data('innings1')
    home_bowling_data = get_bowling_data('innings1')
    away_batting_data = get_batting_data('innings2')
    away_bowling_data = get_bowling_data('innings2')

    # Check if any batting or bowling data is missing
    if not home_batting_data or not home_bowling_data or not away_batting_data or not away_bowling_data:
        print(f"Skipping match with missing batting or bowling data: {scorecard_link}")
        return None

    # Create a new record for the match
    match_record = {
        'Home Team': home_team,
        'Away Team': away_team,
        'Ground': ground_info,
        'Home Score': home_score,
        'Away Score': away_score,
        'Home Batting Data': home_batting_data,
        'Home Bowling Data': home_bowling_data,
        'Away Batting Data': away_batting_data,
        'Away Bowling Data': away_bowling_data
    }

    return match_record


In [66]:
# get game links 
scorecard_links = mine_match_url("https://boldon.play-cricket.com/website/results/5985711")
# get game data
all_match_data = []
for link in scorecard_links:
    match_data = mine_match_data(link)
    all_match_data.append(match_data)
    
print("All match data has been successfully mined and stored.")

https://boldon.play-cricket.com


C:\Users\77977\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


All match data has been successfully mined and stored.


In [64]:
all_match_data

[None,
 {'Home Team': 'Washington CC, Durham - Washington U13 Nova',
  'Away Team': 'Durham City CC - Under 13 Gold',
  'Ground': 'The Harraton Cricket Ground',
  'Home Score': "74-2  (13.1 overs)  't'",
  'Away Score': "73 all out  (18.1 overs)  'b'",
  'Home Batting Data': [{'Name': 'Henry Burns',
    'How Out': '',
    'Bowler': 'b  Sam Hood',
    'Runs': '1',
    '4s': '0',
    '6s': '0',
    'Balls': '5'},
   {'Name': 'Matthew King',
    'How Out': '',
    'Bowler': 'b  Ajith Varghese',
    'Runs': '6',
    '4s': '0',
    '6s': '0',
    'Balls': '25'},
   {'Name': 'Barney Ward†',
    'How Out': '',
    'Bowler': 'b  Ajith Varghese',
    'Runs': '2',
    '4s': '0',
    '6s': '0',
    'Balls': '8'},
   {'Name': 'Jack Hood',
    'How Out': '',
    'Bowler': 'b  Rhys Morgan',
    'Runs': '20',
    '4s': '2',
    '6s': '0',
    'Balls': '24'},
   {'Name': 'James McLaughlin',
    'How Out': 'ct Alex Garthwaite',
    'Bowler': 'b  Jack Wrightson',
    'Runs': '10',
    '4s': '1',
    '6s

In [67]:
import os
import pandas as pd

# Filter out None values
filtered_match_data = [match for match in all_match_data if match is not None]

# Create DataFrame
df_all_matches = pd.DataFrame(filtered_match_data)

# Ensure the score columns are of string type to avoid pandas auto-converting to dates
df_all_matches['Home Score'] = df_all_matches['Home Score'].astype(str)
df_all_matches['Away Score'] = df_all_matches['Away Score'].astype(str)

# Specify the save path
csv_path = r'C:\Users\77977\OneDrive\Desktop\Cricket_data_2023\Boldon_CC_-_Under_13_games.csv'

# Ensure the directory exists, create it if it doesn't
directory = os.path.dirname(csv_path)
if not os.path.exists(directory):
    os.makedirs(directory)

# Save the DataFrame to a CSV file
df_all_matches.to_csv(csv_path, index=False, date_format='string')

print(f"Data saved successfully to {csv_path}")


KeyError: 'Home Score'

In [42]:
from docx import Document

def extract_teams_from_docx(docx_path):
# prepare the word document which contains all the target team names and links
    document = Document(docx_path)
    teams = {}

    for para in document.paragraphs:
        text = para.text.strip()
        if "http" in text:
            try:
                parts = text.split("http", 1)
                team_name = parts[0].strip()
                team_link = "http" + parts[1].strip()
                teams[team_name] = team_link
            except ValueError:
                print(f"Skipping line: {text}")
                continue

    return teams


docx_path = r'C:\Users\77977\OneDrive\Desktop\links.docx'
teams = extract_teams_from_docx(docx_path)

print(teams)


{'Annfield Plain CC - Under 13': 'https://annfieldplain.play-cricket.com/Matches?utf8=%E2%9C%93&tab=Result&selected_season_id=256&seasonchange=f&fixture_month=4&season_id=256&season_id=256&team_id=125598&view_by=year&view_by=year&team_id=125598&search_in=&q%5Bcategory_id%5D=all&q%5Bgender_id%5D=all&home_or_away=both', 'Beamish and E. Stanley CC - Under 13': 'https://beamishestanley.play-cricket.com/Matches?utf8=%E2%9C%93&tab=Result&selected_season_id=256&seasonchange=f&fixture_month=4&season_id=256&season_id=256&team_id=139179&view_by=year&view_by=year&team_id=139179&search_in=&q%5Bcategory_id%5D=all&q%5Bgender_id%5D=all&home_or_away=both', 'Bill Quay CC - Under 13': 'https://billquaycc.play-cricket.com/Matches?utf8=%E2%9C%93&tab=Result&selected_season_id=256&seasonchange=f&fixture_month=4&season_id=256&season_id=256&team_id=329473&view_by=year&view_by=year&team_id=329473&search_in=&q%5Bcategory_id%5D=all&q%5Bgender_id%5D=all&home_or_away=both', 'Bishop Auckland CC - Under 13 B': 'http

In [40]:
def fetch_and_save_team_matches(team_name, matches_link, output_directory):
   
    # Get game links
    scorecard_links = mine_match_url(matches_link)
    
    # Get game data
    all_match_data = []
    for link in scorecard_links:
        match_data = mine_match_data(link)
        if match_data is not None:
            all_match_data.append(match_data)
    
    print(f"All match data has been successfully mined for {team_name}.")
    
    # Filter out None values
    filtered_match_data = [match for match in all_match_data if match is not None]
    
    if filtered_match_data:
        # Create DataFrame
        df_all_matches = pd.DataFrame(filtered_match_data)
        
        # Ensure score columns are of string type
        df_all_matches['Home Score'] = df_all_matches['Home Score'].astype(str)
        df_all_matches['Away Score'] = df_all_matches['Away Score'].astype(str)
        
        # Specify CSV path using team name
        csv_path = os.path.join(output_directory, f"{team_name.replace(' ', '_')}_games.csv")
        
        # Ensure the directory exists
        directory = os.path.dirname(csv_path)
        if not os.path.exists(directory):
            os.makedirs(directory)
        
        # Save DataFrame to CSV file
        df_all_matches.to_csv(csv_path, index=False, date_format='string')
        
        print(f"Data saved successfully to {csv_path}")
    else:
        print(f"No valid match data found for {team_name}.")

def main():
    # Define  team names and links
    output_directory = r'C:\Users\77977\OneDrive\Desktop\Cricket_data_2023'  
    
    # iterate the list
    for team_name, team_link in teams.items():
        fetch_and_save_team_matches(team_name, team_link, output_directory)

if __name__ == "__main__":
    main()

https://murton.play-cricket.com


C:\Users\77977\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Error: Could not find ground info in https://murton.play-cricket.com/website/results/5808764/print
Error: Could not find ground info in https://murton.play-cricket.com/website/results/5808716/print
All match data has been successfully mined for Murton CC - Under 13.
Data saved successfully to C:\Users\77977\OneDrive\Desktop\Cricket_data_2023\Murton_CC_-_Under_13_games.csv
https://peterlee.play-cricket.com


C:\Users\77977\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Skipping match with missing batting or bowling data: https://peterlee.play-cricket.com/website/results/6037812/print
Error: Could not find ground info in https://peterlee.play-cricket.com/website/results/5808765/print
All match data has been successfully mined for Peterlee CC - Under 13.
Data saved successfully to C:\Users\77977\OneDrive\Desktop\Cricket_data_2023\Peterlee_CC_-_Under_13_games.csv
https://philadelphia.play-cricket.com


...

Skipping match with missing batting or bowling data: https://whickham.play-cricket.com/website/results/6047444/print
Skipping match with missing batting or bowling data: https://whickham.play-cricket.com/website/results/6044499/print
Error fetching data from https://whickham.play-cricket.com/website/results/5832768/print: ('Connection aborted.', ConnectionAbortedError(10053, '你的主机中的软件中止了一个已建立的连接。', None, 10053, None))
Retrying... (1/3)
Error: Could not find ground info in https://whickham.play-cricket.com/website/results/5832768/print
Error: Could not find ground info in https://whickham.play-cricket.com/website/results/5832754/print
Error: Could not find ground info in https://whickham.play-cricket.com/website/results/5832764/print
Skipping match with missing batting or bowling data: https://whickham.play-cricket.com/website/results/6014301/print
Error: Could not find ground info in https://whickham.play-cricket.com/website/results/5832758/print
Error: Could not find ground info in ht

C:\Users\77977\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Error: Could not find ground info in https://whitburn.play-cricket.com/website/results/5688035/print
Skipping match with missing batting or bowling data: https://whitburn.play-cricket.com/website/results/5787936/print
Skipping match with missing batting or bowling data: https://whitburn.play-cricket.com/website/results/6037546/print
Skipping match with missing batting or bowling data: https://whitburn.play-cricket.com/website/results/5787943/print
Error: Could not find ground info in https://whitburn.play-cricket.com/website/results/6011986/print
Skipping match with missing batting or bowling data: https://whitburn.play-cricket.com/website/results/5787938/print
Error: Could not find ground info in https://whitburn.play-cricket.com/website/results/5788426/print
All match data has been successfully mined for Whitburn CC - Under 13 A.
Data saved successfully to C:\Users\77977\OneDrive\Desktop\Cricket_data_2023\Whitburn_CC_-_Under_13_A_games.csv
https://willington.play-cricket.com


C:\Users\77977\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Error: Could not find ground info in https://willington.play-cricket.com/website/results/6044509/print
Skipping match with missing batting or bowling data: https://willington.play-cricket.com/website/results/5789399/print
Error fetching data from https://willington.play-cricket.com/website/results/5998980/print: HTTPSConnectionPool(host='willington.play-cricket.com', port=443): Read timed out. (read timeout=10)
Retrying... (1/3)
All match data has been successfully mined for Willington CC - Under 13.
Data saved successfully to C:\Users\77977\OneDrive\Desktop\Cricket_data_2023\Willington_CC_-_Under_13_games.csv
